In [ ]:
import os
os.environ["JAVA_HOME"] = "/opt/java"
os.environ["SPARK_HOME"] = "/opt/spark"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark.sparkContext.addPyFile(r'/opt/airflow/dags/transform_funcs.py')
spark.sparkContext.addPyFile(r'/opt/airflow/dags/dag_initial_loading_ASTRA.py')
spark.sparkContext.addPyFile(r'/opt/airflow/dags/casspark.py')
spark.sparkContext.addPyFile(r'/opt/airflow/dags/tiny_functions.py')
import transform_funcs
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)
import pandas as pd
import datetime
from airflow import DAG
from airflow.operators.python import PythonOperator
from cassandra.cluster import Cluster
from time import sleep
from functools import reduce
from tiny_functions import *

import os
os.environ["JAVA_HOME"] = "/opt/java"
os.environ["SPARK_HOME"] = "/opt/spark"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark.sparkContext.addPyFile(r'/opt/airflow/dags/transform_funcs.py')
spark.sparkContext.addPyFile(r'/opt/airflow/dags/dag_initial_loading_ASTRA.py')
spark.sparkContext.addPyFile(r'/opt/airflow/dags/casspark.py')
spark.sparkContext.addPyFile(r'/opt/airflow/dags/tiny_functions.py')

import transform_funcs
import casspark
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)
# import databricks.koalas as ks
# ks.set_option('compute.ops_on_diff_frames', True)

import pandas as pd
import datetime
#import dateutil
#import pathlib
from airflow import DAG
from airflow.operators.python import PythonOperator
from cassandra.cluster import Cluster
from time import sleep
from functools import reduce
from tiny_functions import *


def connect_to_astra():
    print('ESTABLISHING CONNECTION TO CASSANDRA')
    cluster = Cluster(contact_points=['cassandra'],port=9042)
    session = cluster.connect()
    return session

def lower_col_names(cols):
    new_names = {}
    for x in cols:
        new_names[x] = x.lower()
    return new_names    

/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/airflow/.ivy2/cache
The jars for the packages stored in: /home/airflow/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-93e3f188-0979-4487-8564-39a13755f309;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.2.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.2.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central

22/12/05 05:37:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/05 05:37:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/home/airflow/.local/lib/python3.9/site-packages/airflow/configuration.py:363: FutureWarning: The auth_backends setting in [api] has had airflow.api.auth.backend.session added in the running config, which is needed by the UI. Please update your config before Apache Airflow 3.0.
  warnings.warn(


22/12/05 05:37:12 WARN SparkContext: The path /opt/airflow/dags/transform_funcs.py has been added already. Overwriting of added paths is not supported in the current version.
22/12/05 05:37:12 WARN SparkContext: The path /opt/airflow/dags/dag_initial_loading_ASTRA.py has been added already. Overwriting of added paths is not supported in the current version.
22/12/05 05:37:12 WARN SparkContext: The path /opt/airflow/dags/casspark.py has been added already. Overwriting of added paths is not supported in the current version.
22/12/05 05:37:12 WARN SparkContext: The path /opt/airflow/dags/tiny_functions.py has been added already. Overwriting of added paths is not supported in the current version.


In [ ]:
"""
The function takes a dataframe of users and returns a dataframe with the influencer score for each
user
:param user: the user dataframe
:return: A dataframe with the columns: n_interactions_received, n_interactions_send, fans,
friends_number, Score_influencer, Influencer, user_id
"""
print('READING USER FILE')
user = ps.read_json(r'/opt/data/initial_load/filtered_user.json')
print('DROPPING DUPLICATED ROWS')
user = user.drop_duplicates()
print('NORMALIZING DATES')
user['yelping_since'] = user['yelping_since'].apply(transform_funcs.transform_dates).dt.strftime('%Y-%m-%d')
print('GENERATING INTERACTIONS RECIEVED COLUMN')
user_df = user.copy()



READING USER FILE


22/12/05 05:45:50 WARN MemoryStore: Not enough space to cache rdd_744_9 in memory! (computed 19.4 MiB so far)
22/12/05 05:45:51 WARN MemoryStore: Not enough space to cache rdd_744_10 in memory! (computed 14.9 MiB so far)
22/12/05 05:45:52 WARN MemoryStore: Not enough space to cache rdd_744_8 in memory! (computed 46.4 MiB so far)


22/12/05 05:45:54 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_744_12 in memory.
22/12/05 05:45:54 WARN MemoryStore: Not enough space to cache rdd_744_12 in memory! (computed 384.0 B so far)
22/12/05 05:45:54 WARN MemoryStore: Not enough space to cache rdd_744_11 in memory! (computed 5.3 MiB so far)


22/12/05 05:46:04 WARN MemoryStore: Not enough space to cache rdd_744_14 in memory! (computed 24.7 MiB so far)
22/12/05 05:46:04 WARN MemoryStore: Not enough space to cache rdd_744_15 in memory! (computed 27.2 MiB so far)
[2022-12-05 05:46:08,730] {java_gateway.py:1066} ERROR - KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
[2022-12-05 05:46:08,886] {clientserver.py:543} INFO - Closing down clientserver connection


KeyboardInterrupt: 

In [42]:

print('GENERATING INTERACTIONS RECIEVED COLUMN')
user_df['n_ints_rec'] = user_df[[ 'compliment_hot','compliment_more', 'compliment_profile', 'compliment_cute','compliment_list', 'compliment_note', 'compliment_plain','compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos']].sum(axis=1)
user_df.head(5)



GENERATING INTERACTIONS RECIEVED COLUMN


/opt/spark/python/pyspark/pandas/internal.py:1573 FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.

/opt/spark/python/pyspark/sql/pandas/conversion.py:486 FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.

Py4JJavaError: An error occurred while calling o9332.collectToPython.
: java.lang.ClassCastException: class scala.Tuple8 cannot be cast to class scala.Tuple7 (scala.Tuple8 and scala.Tuple7 are in unnamed module of loader 'app')
	at org.apache.spark.sql.cassandra.execution.CassandraDirectJoinStrategy.apply(CassandraDirectJoinStrategy.scala:27)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:69)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:196)
	at scala.collectio

Py4JJavaError: An error occurred while calling o9378.collectToPython.
: java.lang.ClassCastException: class scala.Tuple8 cannot be cast to class scala.Tuple7 (scala.Tuple8 and scala.Tuple7 are in unnamed module of loader 'app')
	at org.apache.spark.sql.cassandra.execution.CassandraDirectJoinStrategy.apply(CassandraDirectJoinStrategy.scala:27)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:69)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:196)
	at scala.collectio

In [34]:
user_df['n_ints_rec'] = a.copy()

In [35]:
user_df.head(5)

Py4JJavaError: An error occurred while calling o6576.collectToPython.
: java.lang.ClassCastException: class scala.Tuple8 cannot be cast to class scala.Tuple7 (scala.Tuple8 and scala.Tuple7 are in unnamed module of loader 'app')
	at org.apache.spark.sql.cassandra.execution.CassandraDirectJoinStrategy.apply(CassandraDirectJoinStrategy.scala:27)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:69)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:196)
	at scala.collectio

Py4JJavaError: An error occurred while calling o6622.collectToPython.
: java.lang.ClassCastException: class scala.Tuple8 cannot be cast to class scala.Tuple7 (scala.Tuple8 and scala.Tuple7 are in unnamed module of loader 'app')
	at org.apache.spark.sql.cassandra.execution.CassandraDirectJoinStrategy.apply(CassandraDirectJoinStrategy.scala:27)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:69)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:196)
	at scala.collectio

In [ ]:
print('GENERATING INTERACTIONS SEND COLUMN')
user_df['n_interactions_send'] = user_df['useful'] + user_df['funny'] + user_df['cool']
print('GENERATING INFLUENCER COLUMN')
user_df['Influencer'] = user_df['n_ints_rec'] / (1 + user_df['friends_number'] + user_df['fans'])
user_df['Influencer'].fillna(0, inplace = True)
print('GENERATING INFLUENCER SCORE COLUMN')
user_df['Influencer_Score'] = 1 - (1 / (1 + user_df['Influencer']))
print('GENERATING INFLUENCER 2 COLUMN')
user_df['Influencer_2'] = user_df['n_ints_rec'] / (1 + user_df['fans'])
user_df['Influencer_2'].fillna(0, inplace = True)
print('GENERATING INFLUENCER SCORE 2 COLUMN')
user_df['Influencer_Score_2'] = 1 - (1 / (1 + user_df['Influencer_2']))
print('GENERATING NEW DF')
user_df = user_df[['user_id', 'n_ints_rec', 'n_interactions_send', 'fans', 'friends_number',
'Influencer', 'Influencer_Score', 'Influencer_2', 'Influencer_Score_2']]
print('TRANSFORMATIONS FOR USER METRICS DONE')
user_df.rename(columns=lower_col_names(user_df.columns), inplace=True)
print('STABLISHING CONNECTION TO ASTRA')
session = connect_to_astra()

print('DROPPING TABLE IF EXISTS')
session.execute("DROP TABLE IF EXISTS yelp.user_metrics_full;")

print('CREATING TABLE FOR USER METRICS')
session.execute("""
CREATE TABLE IF NOT EXISTS yelp.user_metrics_full(
    user_id text, 
    n_ints_rec int,
    n_interactions_send int,
    fans int,
    friends_number int,
    Influencer float,
    Influencer_Score float,
    Influencer_2 float,
    Influencer_Score_2 float,
    PRIMARY KEY(user_id));
""") #friends list <text>,

sleep(5)
print('UPLOADING DATAFRAME TO CASSANDRA FOR USER METRICS')
user_df.to_spark().write\
.format("org.apache.spark.sql.cassandra")\
.mode('append')\
.options(table="user_metrics_full", keyspace="yelp")\
.save()
print('DONE')